In [1]:
import os
from PIL import Image
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
import imageio
import cv2
from matplotlib.widgets import RectangleSelector
from matplotlib.patches import Rectangle

In [2]:
images_path = r'resized_images'
roi1_folder = r'templates' #-----------------roi1-----------------
roi2_folder = r'medijastinum' #--------------roi2------------------

In [3]:
#---------------------odabrano filtriranje----------------------
def filtriranje(images, method='non_local_means'):
    denoised_images = []
    for img in images:
        denoised_img_nlm = cv2.fastNlMeansDenoising(img, None, h=10, templateWindowSize=7, searchWindowSize=21)
        denoised_img_nlm_bilateral = cv2.bilateralFilter(denoised_img_nlm, d=30, sigmaColor=30, sigmaSpace=30)
        denoised_images.append(denoised_img_nlm_bilateral)
    return denoised_images

In [7]:
# --------------------- ucitavanje slika i sablona ---------------------
def ucitaj_slike(folder, target_size=(512, 512), max_images=500):
    slike = []
    files = [f for f in sorted(os.listdir(folder)) if f.endswith(('png', 'jpg', 'jpeg'))]
    
    files = files[:max_images]
    
    for f in files:
        img_path = os.path.join(folder, f)
        img = np.array(Image.open(img_path).convert('L'))
        #img = cv2.resize(img, target_size)  <------- slike istih dimenzija su sacuvane u folderu resized_images
        slike.append(img)
    return slike

def ucitaj_sablone(folder):
    sabloni = []
    for f in sorted(os.listdir(folder)):
        if f.endswith(('png', 'jpg', 'jpeg')):
            img_path = os.path.join(folder, f)
            img = np.array(Image.open(img_path).convert('L'))
            sabloni.append(img)  # zadrzava originalnu velič=cinu
    return sabloni


In [9]:
slike = ucitaj_slike(images_path) 
roi1 = ucitaj_sablone(roi1_folder)
roi2 = ucitaj_sablone(roi2_folder)
slike_filt = filtriranje(slike)
roi1_filt = filtriranje(roi1)
roi2_filt = filtriranje(roi2)

In [11]:
# -------------------------------------prvi pristup-------------------------------
# --------------------- template matching ---------------------
def pronadji_slican_region(img, sablon):
    h_sablona, w_sablona = sablon.shape[:2]
    h_slike, w_slike = img.shape[:2]

    # ako je sablon veci od slike -> smanjiti
    if h_slike < h_sablona or w_slike < w_sablona:
        scale_factor = min(h_slike / h_sablona, w_slike / w_sablona)
        sablon = cv2.resize(sablon, (int(w_sablona * scale_factor), int(h_sablona * scale_factor)))

    result = cv2.matchTemplate(img, sablon, cv2.TM_CCOEFF_NORMED)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
    start_x, start_y = max_loc
    h, w = sablon.shape[:2]
    return (start_x, start_y, w, h), max_val

#------------------------izdvajanje najboljih roi------------------------

def najbolji_roi(img, sabloni):
    best_roi = None
    best_score = -np.inf
    for sablon in sabloni:
        roi, score = pronadji_slican_region(img, sablon)
        if score > best_score:
            best_score = score
            best_roi = roi
    return best_roi, best_score

# --------------------- crtanje ROI ---------------------
def prikazi_roi(img, roi1, roi2):
    img_color = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    if roi1:
        x, y, w, h = roi1
        cv2.rectangle(img_color, (x, y), (x+w, y+h), (0, 0, 255), 2)  # crveni
    if roi2:
        x, y, w, h = roi2
        cv2.rectangle(img_color, (x, y), (x+w, y+h), (0, 255, 0), 2)  # zeleni
    return img_color

In [15]:
#--------------------------poboljsani pristup------------------------------------
# --------------------- multivelicinska template matching ---------------------
#--------------------------Gausova piramida-----------------------------
def im_pyr_decomp(im, N):
    GPyr = []
    for i in range(N):
        GPyr.append(im) # originalna slika
        g = cv2.pyrDown(im, borderType=cv2.BORDER_REPLICATE)
        im = g 
    Res = im
    return GPyr, Res
    
def najbolji_roi_multiscale(img, sabloni, max_levels=3):
    """
    img: filtrirana slika
    sabloni: lista filtriranih sablona (bez promene dimenzije)
    max_levels: koliko nivoa piramide da se koristi
    """
    best_roi = None
    best_score = -np.inf
    
    GPyr, _ = im_pyr_decomp(img, max_levels) 
    
    for sablon in sabloni:
        for level, img_lvl in enumerate(GPyr):
            roi_lvl, score = pronadji_slican_region(img_lvl, sablon)
            
            if score > best_score:
                best_score = score
                scale_factor = 2 ** level
                x, y, w, h = roi_lvl
                # skaliramo koordinate dok je velicina roi-a ista
                best_roi = (x * scale_factor, y * scale_factor, w, h)
    
    return best_roi, best_score


In [17]:
# --------------------- pravljenje gifova---------------------------
def dodaj_score_text(img, score1, score2, metod):
    img_copy = img.copy()
    cv2.putText(img_copy, f"{metod} - ROI1: {score1:.3f}", 
                (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    cv2.putText(img_copy, f"ROI2: {score2:.3f}", 
                (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    return img_copy
    
def process_all_images(output_gif_path1, output_gif_path2):
    
    sabloni1 = roi1_filt
    sabloni2 = roi2_filt
    if not sabloni1 or not sabloni2:
        print("Nema učitanih šablona.")
        return
    processed_images1 = []
    processed_images2 = []
    for image_filt in slike_filt:
        # najbolji ROI iz obe grupe
        roi1_prvi_pristup, score1_prvi_pristup = najbolji_roi(image_filt, sabloni1)
        roi2_prvi_pristup, score2_prvi_pristup = najbolji_roi(image_filt, sabloni2)
        roi1_drugi_pristup, score1_drugi_pristup = najbolji_roi_multiscale(image_filt, sabloni1)
        roi2_drugi_pristup, score2_drugi_pristup = najbolji_roi_multiscale(image_filt, sabloni2)
        
        # Crtanje ROI regiona
        img_with_rois1 = prikazi_roi(image_filt, roi1_prvi_pristup, roi2_prvi_pristup)
        img_with_rois2 = prikazi_roi(image_filt, roi1_drugi_pristup, roi2_drugi_pristup)
        
        # Dodavanje score teksta
        img_with_scores1 = dodaj_score_text(img_with_rois1, score1_prvi_pristup, score2_prvi_pristup, "Prvi pristup")
        img_with_scores2 = dodaj_score_text(img_with_rois2, score1_drugi_pristup, score2_drugi_pristup, "Multiscale")
        
        processed_images1.append(Image.fromarray(cv2.cvtColor(img_with_scores1, cv2.COLOR_BGR2RGB)))
        processed_images2.append(Image.fromarray(cv2.cvtColor(img_with_scores2, cv2.COLOR_BGR2RGB)))
        
    if processed_images1:
        processed_images1[0].save(
            output_gif_path1,
            save_all=True,
            append_images=processed_images1[1:],
            duration=500,
            loop=0
        )
        print(f"GIF sačuvan u {output_gif_path1}")
    if processed_images2:
        processed_images2[0].save(
            output_gif_path2,
            save_all=True,
            append_images=processed_images2[1:],
            duration=500,
            loop=0
        )
        print(f"GIF sačuvan u {output_gif_path2}")
    else:
        print("Nema obrađenih slika za GIF.")




In [19]:
output_gif_path1 = 'slike_sa_roi_prvi_pristup.gif'
output_gif_path2 = 'slike_sa_roi_multiscale.gif'
process_all_images(output_gif_path1, output_gif_path2)

GIF sačuvan u slike_sa_roi_prvi_pristup.gif
GIF sačuvan u slike_sa_roi_multiscale.gif
